In [2]:
# import packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from itertools import chain
df = pd.read_csv('../res/input0130.csv')

In [4]:
scaler = StandardScaler()
starter = range(240,len(df)-1000,250)
all_results = []
all_evas = []
y_pred_clf = []
all_x_train = []
all_x_test = []
all_y_train = []
all_y_test = []
for train_begin in starter:
    train_end = train_begin + 750
    test_end = train_end + 250
    train_set = df[['log_r','label']][train_begin:train_end].reset_index()
    x_train, y_train = [], []
    x_train_set = list(chain.from_iterable(scaler.fit_transform(train_set['log_r'].values.reshape(-1,1))))
    for i in range(240,len(x_train_set)):
        x_train.append(x_train_set[i-240:i])
        y_train.append(train_set['label'][i])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1)) 
    
    # model = lstm_model()
    # model.fit(x_train,y_train,epochs=100,callbacks=[EarlyStopping(monitor='loss',patience=10)])

    x_test, y_test = [], []
    test_set = df[['log_r','label']][train_end-240:test_end].reset_index()
    x_test_set = list(chain.from_iterable(scaler.transform(test_set['log_r'].values.reshape(-1,1))))
    for i in range(240,len(x_test_set)):
        x_test.append(x_test_set[i-240:i])
        y_test.append(test_set['label'][i])
    x_test, y_test = np.array(x_test), np.array(y_test)
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1)) 
    all_x_train.append(x_train)
    all_y_train.append(y_train)
    all_x_test.append(x_test)
    all_y_test.append(y_test)

    # all_results.append(model.predict(x_test))
    # all_evas.append(model.evaluate(x_test, y_test))
    # y_pred_clf.append(model.predict_classes(x_test))

In [6]:
from tqdm import tqdm
from keras.models import load_model, Model
from sa import *
from utils import *

In [7]:
model = load_model('lstm_r.h5')
default_upper_bound = 2000
default_n_bucket = 1000
default_n_classes = 2
class Args(): #创建一个类
    def __init__(self): #定义初始化信息。
        self.is_classification = True
        self.save_path = './tmp/'
        self.d = 'lstm_r'
        self.num_classes = 2
        self.lsa = True
        self.dsa = True
        self.target = 'none'
        self.batch_size = 128
        self.var_threshold = 1e-5
        self.upper_bound = 2000
        self.n_bucket = 1000
        self.is_classification = True
args = Args()
model.summary()

KeyError: 0

In [5]:
layer_names = ['activation_1']

In [7]:
test_lsa = fetch_lsa(model, x_train, x_test, "test", layer_names, args)
target_lsa = fetch_lsa(model, x_train, x_test, "target", layer_names, args)
# target_lsa = fetch_lsa(model, x_train, x_target, args.target, layer_names, args)
target_cov = get_sc(
    np.amin(target_lsa), default_upper_bound, default_n_bucket, target_lsa
)

auc = compute_roc_auc(test_lsa, target_lsa)
print(infog("ROC-AUC: " + str(auc * 100)))

print(infog("LSA coverage: " + str(target_cov)))

Found saved train ATs, skip serving
Found saved test ATs, skip serving


kde:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)